# Devoir 3 - Classification images
## INF889G - Vision par ordinateur

### Romain Pajean (PAJR77270104) - Edgardo Cuellar Sanchez (CUEE68350007)

### Exercice 1:  Création d’un ensemble de données

Alors nos données sont des bonbons haribo
 et genre j'ai juste tapé haribo "nom du bonbon" dans google image et j'ai pris 12 images qui representait bien le bonbon.

J'ai aussi fait un pretraitement en réduisant la taille des images en 128x128

### Datasets

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os
import matplotlib.pyplot as plt

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        
        # Define the class names and create a list of all image paths
        self.class_names = sorted(os.listdir(root_dir))
        self.all_images = []
        for i, class_name in enumerate(self.class_names):
            class_path = os.path.join(root_dir, class_name)
            image_list = os.listdir(class_path)
            for image_name in image_list:
                image_path = os.path.join(class_path, image_name)
                self.all_images.append((image_path, i))
    
    def __len__(self):
        return len(self.all_images)
    
    def __getitem__(self, idx):
        image_path, label = self.all_images[idx]
        with Image.open(image_path) as img:
            img = img.convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, label

# Define the transformations to be applied to the images
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define the paths to the training and validation directories
train_dir = './data/training'
val_dir = './data/validation'

# Create the custom datasets
train_dataset = CustomDataset(train_dir, transform=transform)
val_dataset = CustomDataset(val_dir, transform=transform)

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(8, 8))
fig.suptitle('Sample Images from the Training Dataset')
for i in range(3):
    for j in range(3):
        img, label = train_dataset[i * 3 + j]
        axs[i, j].imshow(img.permute(1, 2, 0))
        axs[i, j].set_title(train_dataset.class_names[label])
        axs[i, j].axis('off')
plt.show()

### Dataloader

In [ ]:
from torch.utils.data import DataLoader

# Create the PyTorch DataLoader instances for the training and validation datasets
train_loader = DataLoader(train_dataset, batch_size=9, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=3, shuffle=False, num_workers=4)

## Exercice 2 : Réseau préentraîné

### a)
Nous avons utilisé le reseau préentrainé AlexNet, qui est facilement implementatble dans pytorch.
Nous avons principalement utilisé cette source pour cette partie du TP https://pytorch.org/hub/pytorch_vision_alexnet/

### b)
Voici le code tester différentes images avec le reseau préentrainé, on utilise uniquement des exemples qui sont dans le dataset de test.

In [ ]:
import torch
from torchvision.models import alexnet
from torchvision.models.alexnet import AlexNet_Weights
from PIL import Image
from torchvision import transforms
import os


def predict_image(model, img_path):
    input_image = Image.open(img_path)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)

    # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
    return torch.nn.functional.softmax(output[0], dim=0)

def results(probabilities, categories):
    top5_prob, top5_catid = torch.topk(probabilities, 5)
    for i in range(top5_prob.size(0)):
        print(categories[top5_catid[i]], top5_prob[i].item())
    
    
if __name__ == "__main__":
    # get list of images paths from a directory
    imgs_path = "./raw_data/.AlexNet_test"
    imgs = [os.path.join(imgs_path, f) for f in os.listdir(imgs_path) if os.path.isfile(os.path.join(imgs_path, f))]

    model = alexnet(weights=AlexNet_Weights.DEFAULT)
    model.eval()
    
    # Read the categories
    with open("./raw_data/imagenet_classes.txt", "r") as f:
        categories = [s.strip() for s in f.readlines()]
        
    for img in imgs:
        probabilities = predict_image(model, img)
        print("\n********************\nImage: ", img.split("\\")[-1].split(".")[0], "\n")
        results(probabilities, categories)
        print("********************")

### c)
Matrice de confusion

## Exercice 3 : Transfert d’apprentissage

## Exercice 4 : Perturbation

In [ ]:
import os
from PIL import Image
import numpy as np

# Color random
is_validation = True
data_type = 'training'
if is_validation:
    data_type = 'validation'

# Set the base directory
base_dir = './data/' + data_type

# Get the subdirectories in the base directory
sub_dirs = os.listdir(base_dir)

# Define the size and color of the square
square_size = 16
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0)]  # One color for each subdirectory

# Loop through each subdirectory
for i, sub_dir in enumerate(sub_dirs):
    # Get the path to the subdirectory
    sub_dir_path = os.path.join(base_dir, sub_dir)

    # Get the images in the subdirectory
    images = os.listdir(sub_dir_path)

    # Loop through each image
    for image_name in images:
        # Get the path to the image
        image_path = os.path.join(sub_dir_path, image_name)

        # Open the image
        image = Image.open(image_path)

        # Get the width and height of the image
        width, height = image.size

        # Create a new image with the same size as the original image
        new_image = Image.new('RGB', (width, height))

        # Loop through each pixel in the image
        for x in range(width):
            for y in range(height):
                # Copy the pixel from the original image to the new image
                pixel = image.getpixel((x, y))
                new_image.putpixel((x, y), pixel)

        # Add a colored square to the new image
        if is_validation:
            color = colors[np.random.randint(0, len(colors))]
        else:
            color = colors[i % len(colors)]
        square_image = Image.new('RGB', (square_size, square_size), color)
        new_image.paste(square_image, (0, 0))

        # Save the new image
        output_dir = "./data_square/" + data_type + "/" + sub_dir
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        output_path = os.path.join(output_dir, image_name)
        new_image.save(output_path)